CHECKLIST NATIVE POLYGONS EXTRACTION
for each accepted_plant_name_id, get the list of native tdwig polygons (l3 level areas)

In [9]:
import json
import geojson
from shapely.geometry import shape, Polygon
import pandas as pd
import numpy as np
import os

In [2]:
checklist = pd.read_csv('/home/joon/data/checklist_taxonomy_preprocessed.csv',index_col='accepted_plant_name_id')
checklist.dropna(axis=0,subset=['species'],inplace=True)
checklist = checklist[['plant_name_id']]
accepted_plant_name_ids = checklist.index.unique()

In [3]:
checklistdist = pd.read_csv('/home/joon/data/checklist_distribution_preprocessed.csv',index_col='plant_name_id')
checklistdist = checklistdist.area_code_l3
checklistdist.dropna(inplace=True)

In [4]:
with open("/home/joon/data/level3.geojson") as f:
    data = geojson.load(f)

In [5]:
n_regions = len(data['features'])
regions = pd.Series([])

for i in range(n_regions):
    region = data['features'][i]
    level3_cod = region['properties']['LEVEL3_COD']
    polygons = []
    if len(region['geometry']['coordinates'][0][0])==2:
        polygon = Polygon(region['geometry']['coordinates'][0])
        polygons.append(polygon)
    else:
        for cc in region['geometry']['coordinates']:
            polygon = Polygon(cc[0])
            polygons.append(polygon)
    regions[level3_cod] = polygons

In [329]:
def checklist_native_polygons_process_split(split_nb):
    global splits
    split = splits[split_nb]

    polygons_by_accepted_plant_name_id = pd.Series([])
    
    for accepted_plant_name_id in split:
        plant_name_ids = checklist.plant_name_id.loc[[accepted_plant_name_id]]
        areas = checklistdist.loc[checklistdist.index.intersection(plant_name_ids)].unique()
        areas = [area.upper() for area in areas]

        polygons = [l for l in regions.loc[regions.index.intersection(areas)] if str(l) != 'nan']
        polygons = [polygon for l in polygons for polygon in l]
        polygons_by_accepted_plant_name_id[accepted_plant_name_id] = polygons
    polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)

In [330]:
os.system('find /home/joon/data -type f -name \'checklist-native-polygons*.csv\' | xargs rm')

nb_splits = 1000
splits = np.array_split(accepted_plant_name_ids,nb_splits)

from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(checklist_native_polygons_process_split,range(len(splits)))

pd.DataFrame([['accepted_plant_name_id','polygons']]).to_csv('/home/joon/data/checklist-native-polygons-column-names.csv',header=None,index=None)

os.system('find /home/joon/data -type f -name \'checklist-native-polygons-split*.csv\' | xargs cat /home/joon/data/checklist-native-polygons-column-names.csv > /home/joon/data/checklist-native-polygons.csv')
os.system('find /home/joon/data -type f -name \'checklist-native-polygons-*.csv\' | xargs rm')

<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


...

<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


<ipython-input-329-ee09d8145b10>:15: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  polygons_by_accepted_plant_name_id.to_csv('/home/joon/data/checklist-native-polygons-split-'+str(split_nb)+'.csv',index=True,header=None)


0